In [1]:
!pip install fancyimpute mrmr-selection skfeature-chappers

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29880 sha256=ac2dc0beda3912f542a6d317883c67fdd5f932a3f4c136849b63e67fbc7baea3
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=3504a74aa70d462e886ee34e38c706f741fedf0de7c24752600b012a75a1d759
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, LogisticRegression, LassoCV, LinearRegression
from sklearn.feature_selection import mutual_info_classif, chi2, RFE
from mlxtend.feature_selection import SequentialFeatureSelector
from mrmr import mrmr_classif
from fancyimpute import IterativeImputer


import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [3]:
# Open and load the file from Drive
directory = '/content/drive/MyDrive/Colab Notebooks/Project/'
file_path = directory + 'Book2.csv'
with open(file_path) as f:
    data = pd.read_csv(f)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Columns: 188 entries, Samples to Val
dtypes: float64(185), int64(1), object(2)
memory usage: 138.2+ KB


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,Prostate cancer,AA,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,Prostate cancer,EA,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,Case control,EA,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,Case control,EA,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,Case control,EA,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0


In [4]:
df = data.copy()

# shuffling the df
shuffled_df = shuffle(df, random_state=42)

# Encoding catergorical data to numerical data
df['Race'] = df['Race'].map({'EA': 0, 'AA': 1})
df['Status'] = df['Status'].map({'Case control': 0, 'Prostate cancer': 1})

df.head()


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,1,1,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,1,0,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,0,0,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,0,0,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,0,0,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0


In [5]:
# Check for NaN values
nan_check = df.isna().sum()
print(nan_check[nan_check > 0])

# Check total number of NaN values in df
total_nan = df.isna().sum().sum()
print("Total number of NaN values in the DataFrame (Before):", total_nan)

df.tail(10)

PC aa C34:4    1
PC aa C36:0    1
PC aa C36:1    1
PC aa C36:2    1
PC aa C36:3    1
              ..
Ser            1
Thr            2
Trp            1
Tyr            1
Val            1
Length: 160, dtype: int64
Total number of NaN values in the DataFrame (Before): 170


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
84,20049,0,1,63.0,1.93,0.918,19.30,13.20,29.7,1.95,...,229.0,36.7,84.3,98.5,325.0,143.0,221.0,68.8,129.0,368.0
85,20051,0,1,90.3,2.41,1.470,26.10,14.80,22.3,1.79,...,307.0,40.1,123.0,82.7,209.0,134.0,167.0,81.1,101.0,340.0
86,20070,0,1,74.0,2.26,0.933,20.70,15.00,17.2,1.43,...,248.0,33.6,56.5,69.8,219.0,103.0,135.0,47.8,87.0,243.0
87,20078,0,1,109.5,2.36,1.635,35.75,16.85,24.7,1.87,...,108.0,10.5,65.0,37.2,65.4,50.6,NaN,23.5,31.7,107.0
88,20099,0,1,118.0,3.68,2.090,32.30,19.50,27.8,2.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,20103,0,1,137.0,3.00,2.390,47.20,23.50,41.5,2.44,...,207.0,22.9,99.1,68.5,200.0,98.0,111.0,60.4,63.1,213.0
90,20319,0,1,182.0,4.81,2.470,52.60,27.80,46.2,2.81,...,389.0,49.3,201.0,158.0,514.0,136.0,204.0,108.0,140.0,495.0
91,20323,0,1,150.0,3.46,2.260,55.00,29.70,61.0,3.81,...,312.0,29.5,143.0,117.0,330.0,94.6,155.0,87.4,75.2,348.0
92,20361,0,1,133.0,2.74,2.130,45.00,19.80,39.9,2.46,...,300.0,35.0,151.0,87.1,510.0,163.0,247.0,73.0,105.0,324.0
93,20752,0,1,212.0,6.19,2.730,55.10,30.60,40.8,3.61,...,355.0,27.9,170.0,113.0,328.0,153.0,187.0,84.3,86.6,450.0


In [6]:
df = df.drop(columns=['Samples'])

In [7]:
# # Visualize missing data pattern
# msno.matrix(df)
# plt.show()

# # Check for any particular patterns in missing data
# msno.heatmap(df)
# plt.show()


In [32]:
# Split data into train (80%) and test (20%)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=43)

# # Remove some values from the training data for testing imputation
train_data_with_missing = train_data.copy()

columns_to_exclude = ['Status', 'Race']

# Get the list of columns where missing values will be introduced
columns_to_modify = train_data_with_missing.columns.difference(columns_to_exclude)

# Introduce missing values only in the selected columns
train_data_with_missing.loc[::10, columns_to_modify] = pd.NA


In [15]:
# # Split data into train (80%) and test (20%)
# train_data, test_data = train_test_split(df, test_size=0.2, random_state=43)

# # Copy the training data
# train_data_with_missing = train_data.copy()

# # Columns to exclude from missing value introduction
# columns_to_exclude = ['Status', 'Race']
# columns_to_modify = train_data_with_missing.columns.difference(columns_to_exclude)

# # Create a mask to keep track of where we introduce missing values
# introduced_missing_mask = pd.DataFrame(False, index=train_data_with_missing.index, columns=train_data_with_missing.columns)
# introduced_missing_mask.loc[::10, columns_to_modify] = True

# # Introduce missing values only in the selected columns (every 10th row)
# train_data_with_missing = train_data_with_missing.mask(introduced_missing_mask, pd.NA)


In [16]:
train_data_with_missing.tail(20)

,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
4,0,0,57.8,1.78,0.781,18.0,12.1,13.2,1.48,5.43,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.8,115.0
84,0,1,63.0,1.93,0.918,19.3,13.2,29.7,1.95,6.61,...,229.0,36.7,84.3,98.5,325.0,143.0,221.0,68.8,129.0,368.0
79,1,1,99.4,2.52,2.920,29.3,17.1,20.4,1.80,6.73,...,345.0,46.0,100.0,127.0,276.0,145.0,193.0,88.7,138.0,374.0
34,0,0,99.1,2.84,1.680,26.9,24.7,47.8,1.94,11.90,...,296.0,37.5,171.0,96.4,320.0,140.0,205.0,91.7,80.2,298.0
65,1,0,165.0,4.64,3.030,51.0,32.3,43.5,2.79,8.40,...,238.0,29.0,109.0,84.1,235.0,117.0,158.0,121.0,90.2,308.0
75,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,0,1,114.0,2.75,1.240,30.3,22.2,34.5,2.26,9.99,...,251.0,45.5,136.0,104.0,395.0,119.0,182.0,70.2,103.0,409.0
35,0,1,99.9,3.52,1.740,33.3,26.1,31.4,1.83,9.00,...,331.0,46.8,131.0,114.0,327.0,164.0,236.0,87.9,132.0,381.0
92,0,1,133.0,2.74,2.130,45.0,19.8,39.9,2.46,11.40,...,300.0,35.0,151.0,87.1,510.0,163.0,247.0,73.0,105.0,324.0
30,0,0,124.0,3.51,2.570,37.5,22.7,30.9,1.39,5.53,...,337.0,31.4,168.0,88.8,436.0,169.0,179.0,75.4,94.7,299.0


#NaN Value Treatment

MICE (Linear)

In [33]:
mice_linear_imputer = IterativeImputer(estimator=LinearRegression(), max_iter=10, random_state=42)
df_linear_imputed = mice_linear_imputer.fit_transform(train_data_with_missing)

Function for Correlation-Based Imputation

In [34]:
def correlation_based_imputation(df):
    for column in df.columns:
        # Check if there are missing values in the column
        if df[column].isnull().sum() > 0:

            # Find the most correlated feature with the current column
            correlated_feature = df.corr()[column].dropna().idxmax()
            # print(f"Imputing {column} using {correlated_feature}")

            # Impute missing values in the correlated feature using mean (or other methods)
            df[correlated_feature].fillna(df[correlated_feature].mean(), inplace=True)

            # Train linear regression model on non-missing rows
            non_missing_rows = df[df[column].notnull()]
            model = LinearRegression()
            model.fit(non_missing_rows[[correlated_feature]], non_missing_rows[column])

            # Predict missing values for rows where the current column has NaNs
            missing_rows = df[df[column].isnull()]
            if not missing_rows.empty:
                predicted_values = model.predict(missing_rows[[correlated_feature]])
                df.loc[df[column].isnull(), column] = predicted_values
    return df

df_corr_imputed = correlation_based_imputation(train_data_with_missing)


KNN

In [35]:
mice_knn_imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=5, weights='uniform'), max_iter=10, random_state=42)
df_knn_imputed = mice_knn_imputer.fit_transform(train_data_with_missing)


Ridgde

In [36]:
mice_ridge_imputer = IterativeImputer(estimator=Ridge(alpha=1.0), max_iter=10, random_state=42)
df_ridge_imputed = mice_ridge_imputer.fit_transform(train_data_with_missing)


Lasso

In [37]:
mice_lasso_imputer = IterativeImputer(estimator=Lasso(alpha=0.1), max_iter=10, random_state=42)
df_lasso_imputed = mice_lasso_imputer.fit_transform(train_data_with_missing)


ElasticNet

In [38]:
mice_elasticnet_imputer = IterativeImputer(estimator=ElasticNet(alpha=1.0, l1_ratio=0.5), max_iter=10, random_state=42)
df_elasticnet_imputed = mice_elasticnet_imputer.fit_transform(train_data_with_missing)


XGBRegressor

In [39]:
mice_xgb_imputer = IterativeImputer(estimator=XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3), max_iter=10, random_state=42)
df_xgb_imputed = mice_xgb_imputer.fit_transform(train_data_with_missing)


SVR

In [40]:
mice_svr_imputer = IterativeImputer(estimator=SVR(C=1.0, epsilon=0.1), max_iter=10, random_state=42)
df_svr_imputed = mice_svr_imputer.fit_transform(train_data_with_missing)


BayesianRidge

In [41]:
mice_bayesianridge_imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)
df_bayesianridge_imputed = mice_bayesianridge_imputer.fit_transform(train_data_with_missing)


DecisionTreeRegressor

In [42]:
mice_tree_imputer = IterativeImputer(estimator=DecisionTreeRegressor(max_depth=5), max_iter=10, random_state=42)
df_tree_imputed = mice_tree_imputer.fit_transform(train_data_with_missing)


GradientBoostingRegressor

In [43]:
mice_gb_imputer = IterativeImputer(estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3), max_iter=10, random_state=42)
df_gb_imputed = mice_gb_imputer.fit_transform(train_data_with_missing)


RandomForestRegressor

In [44]:
mice_rf_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=100, max_depth=5), max_iter=10, random_state=42)
df_rf_imputed = mice_rf_imputer.fit_transform(train_data_with_missing)


PolynomialRegression

In [45]:
from sklearn.preprocessing import PolynomialFeatures

# Define a custom estimator that uses polynomial regression
class PolynomialRegressionImputer:
    def __init__(self, degree=2):
        self.degree = degree
        self.poly = PolynomialFeatures(degree=self.degree)
        self.model = LinearRegression()

    def fit(self, X, y):
        # Transform features into polynomial terms
        X_poly = self.poly.fit_transform(X)
        self.model.fit(X_poly, y)
        return self

    def predict(self, X):
        # Transform features into polynomial terms for prediction
        X_poly = self.poly.transform(X)
        return self.model.predict(X_poly)

    # Implementing get_params to make the class compatible with scikit-learn
    def get_params(self, deep=True):
        return {"degree": self.degree}

    # Implementing set_params to make the class compatible with scikit-learn
    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self.poly = PolynomialFeatures(degree=self.degree)
        return self

# Apply MICE using polynomial regression
polynomial_imputer = IterativeImputer(estimator=PolynomialRegressionImputer(degree=2), max_iter=10, random_state=42)

# Perform imputation
df_poly_imputed = polynomial_imputer.fit_transform(train_data_with_missing)


In [ ]:
# List of imputed data arrays and their respective names
imputed_data = [
    ('MICE Linear', df_linear_imputed),
    ('MICE Ridge', df_ridge_imputed),
    ('MICE ElasticNet', df_elasticnet_imputed),
    ('MICE XGB', df_xgb_imputed),
    ('MICE SVR', df_svr_imputed),
    ('MICE KNN', df_knn_imputed),
    ('MICE BayesianRidge', df_bayesianridge_imputed),
    ('MICE Lasso', df_lasso_imputed),
    ('MICE Tree', df_tree_imputed),
    ('MICE GB', df_gb_imputed),
    ('MICE RF', df_rf_imputed),
    ('Correlation-Based', df_corr_imputed),
    ('MICE Poly', df_poly_imputed),
]


# Function to calculate RMSE, print results, and return the imputed DataFrames
def calculate_rmse(original, imputed_data):
    results = {}

    for method, imputed_array in imputed_data:
        # Convert numpy array to DataFrame
        imputed_df = pd.DataFrame(imputed_array, columns=original.columns)

        # Ensure original and imputed arrays have the same shape
        assert original.shape == imputed_df.shape, "Original and imputed arrays must have the same shape"

        # Mask arrays to exclude NaN values
        mask = ~np.isnan(original.values)

        # Compute RMSE only for non-NaN entries
        rmse = np.sqrt(mean_squared_error(original.values[mask], imputed_df.values[mask]))

        print(f"{method} Imputation RMSE: {rmse:.4f}")

        # Store the DataFrame and RMSE in the results dictionary
        results[method] = {
            'imputed_df': imputed_df,
            'rmse': rmse
        }

    return results


imputation_results = calculate_rmse(train_data, imputed_data)


In [30]:
# # List of imputed data arrays and their respective names
# imputed_data = [
#     ('MICE Linear', df_linear_imputed),
#     ('MICE Ridge', df_ridge_imputed),
#     ('MICE ElasticNet', df_elasticnet_imputed),
#     ('MICE XGB', df_xgb_imputed),
#     ('MICE SVR', df_svr_imputed),
#     ('MICE KNN', df_knn_imputed),
#     ('MICE BayesianRidge', df_bayesianridge_imputed),
#     ('MICE Lasso', df_lasso_imputed),
#     ('MICE Tree', df_tree_imputed),
#     ('MICE GB', df_gb_imputed),
#     ('MICE RF', df_rf_imputed),
#     ('Correlation-Based', df_corr_imputed),
#     ('MICE Poly', df_poly_imputed),
# ]


# # Function to calculate RMSE, print results, and return the imputed DataFrames
# # Function to calculate RMSE for introduced missing values
# def calculate_rmse(train_data, imputed_data, missing_mask):
#     results = {}

#     for method, imputed_array in imputed_data:
#         # Convert numpy array to DataFrame
#         imputed_df = pd.DataFrame(imputed_array, columns=train_data.columns)

#         # Ensure original and imputed arrays have the same shape
#         assert train_data.shape == imputed_df.shape, "Original and imputed arrays must have the same shape"

#         # Calculate RMSE only for values where we introduced missing data
#         mask = missing_mask.values  # Convert mask DataFrame to boolean NumPy array
#         rmse = np.sqrt(np.nanmean((train_data.values[mask] - imputed_df.values[mask]) ** 2))

#         print(f"{method} Imputation RMSE: {rmse:.4f}")

#         # Store the DataFrame and RMSE in the results dictionary
#         results[method] = {
#             'imputed_df': imputed_df,
#             'rmse': rmse
#         }

#     return results

# # Call the function and pass the missing_mask to only calculate RMSE for introduced missing values
# imputation_results = calculate_rmse(train_data, imputed_data, introduced_missing_mask)


MICE Linear Imputation RMSE: 253.2171
MICE Ridge Imputation RMSE: 259.0006
MICE ElasticNet Imputation RMSE: 259.0006
MICE XGB Imputation RMSE: 259.0006
MICE SVR Imputation RMSE: 259.0006
MICE KNN Imputation RMSE: 259.0006
MICE BayesianRidge Imputation RMSE: 259.0006
MICE Lasso Imputation RMSE: 259.0006
MICE Tree Imputation RMSE: 259.0006
MICE GB Imputation RMSE: 259.0006
MICE RF Imputation RMSE: 259.0006
Correlation-Based Imputation RMSE: 259.0006
MICE Poly Imputation RMSE: 259.0006


# FIlling NaN values

In [ ]:
# Using SVR for NaN value treatment
mice_svr_imputer = IterativeImputer(estimator=SVR(C=1.0, epsilon=0.1), max_iter=10, random_state=42)
df_svr_imputed = mice_svr_imputer.fit_transform(df)

imputed_df = pd.DataFrame(df_svr_imputed, columns=df.columns)
imputed_df.tail(20)

In [ ]:
# Check for NaN values
nan_check = imputed_df.isna().sum()
print(nan_check[nan_check > 0])

# Check total number of NaN values in df
total_nan = imputed_df.isna().sum().sum()
print("Total number of NaN values in the DataFrame (after):", total_nan)

# df.tail(10)

In [ ]:
# Filter rows where Status is "Prostate cancer" (1)
df_race = imputed_df[imputed_df['Status'] == 1]
df_race.info()
df_race.head()

In [ ]:
X = df_race.drop(columns=['Race', 'Status'])
y = df_race['Race']

# feature selection

Mututal Inbformation/information gain

In [ ]:
# Compute mutual information
mi = mutual_info_classif(X, y)

# Create a Series of the results and select top 20 features
mi_df = pd.Series(mi, index=X.columns).nlargest(20)

# Plot the top 20 features with the highest mutual information
mi_df.sort_values(ascending=True).plot(kind='barh', color='blue')
plt.title('Top 20 Features by Mutual Information (Ascending Order)')
plt.xlabel('Mutual Information Score')
plt.ylabel('Features')
plt.show()

# Select the top 20 features
top_20_features_mi = mi_df.index

X_train, X_test, y_train, y_test = train_test_split(X[top_20_features_mi], y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_mi = rf.predict(X_test)
accuracy_mi = accuracy_score(y_test, y_pred_mi)

print("Accuracy using Mutual Information selected features:", accuracy_mi)
print(mi_df)

Chi Sqaure test

In [ ]:
# Scale the features to non-negative values
X_scaled = MinMaxScaler().fit_transform(X)

# Compute chi-square statistics
chi_scores, p_values = chi2(X_scaled, y)

# Create a Series of the chi-square scores and select the top 20 features
chi2_df = pd.Series(chi_scores, index=X.columns).nlargest(20)

# Plot the top 20 features based on chi-square scores
chi2_df.sort_values(ascending=True).plot(kind='barh', color='blue')
plt.title('Top 20 Features by Chi-Square Score (Ascending Order)')
plt.xlabel('Chi-Square Score')
plt.ylabel('Features')
plt.show()

# Select the top 20 features
top_20_features_chi2 = chi2_df.index

X_train, X_test, y_train, y_test = train_test_split(X[top_20_features_chi2], y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_chi2 = rf.predict(X_test)
accuracy_chi2 = accuracy_score(y_test, y_pred_chi2)

print("Accuracy using Chi-Square selected features:", accuracy_chi2)
print(chi2_df)

Pearson Correlation Coefficient

In [ ]:
# Calculate Pearson correlation with respect to the target variable (Race)
pearson_corr = df_race.corr()['Race'].drop('Race')

# Sort by absolute correlation and get the top 20 features
top_20_pearson = pearson_corr.abs().nlargest(20)

# Plotting the top 20 features based on Pearson correlation in blue color
plt.figure(figsize=(10, 6))
top_20_pearson.plot(kind='barh', color='blue')
plt.title('Top 20 Features by Pearson Correlation (Absolute Values)')
plt.xlabel('Pearson Correlation')
plt.ylabel('Features')
plt.show()

# Selecting the top 20 features for model training
top_20_features_pearson = top_20_pearson.index

X_train, X_test, y_train, y_test = train_test_split(X[top_20_features_pearson], y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_prsn = rf.predict(X_test)
accuracy_prsn = accuracy_score(y_test, y_pred_prsn)
print(f'Accuracy using top 20 Pearson correlated features: {accuracy_prsn:.4f}')
print(top_20_pearson)

Spearman Correlation Coefficient

In [ ]:
# Calculate Spearman correlation with respect to the target variable (Race)
spearman_corr = df_race.corr(method='spearman')['Race'].drop('Race')

# Sort by absolute correlation and get the top 20
top_20_spearman = spearman_corr.abs().nlargest(20)

# Plotting the top 20 features based on Spearman correlation
plt.figure(figsize=(10, 6))
top_20_spearman.plot(kind='barh', color='blue')
plt.title('Top 20 Features by Spearman Correlation (Absolute Values)')
plt.xlabel('Spearman Correlation')
plt.ylabel('Features')
plt.show()

# Selecting the top 20 features for model training
top_20_features = top_20_spearman.index

X_train, X_test, y_train, y_test = train_test_split(X[top_20_features], y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_sprmn = rf.predict(X_test)
accuracy_sprmn = accuracy_score(y_test, y_pred_sprmn)

print(f'Accuracy using top 20 Spearman correlated features: {accuracy_sprmn:.4f}')
print(top_20_spearman)


Mean Absolute Difference (MAD)

In [ ]:
# Calculate Mean Absolute Difference (MAD) for each feature with respect to the target variable
mad_values = X.apply(lambda x: (x - y).abs().mean())

# Select top 20 features in descending order
top_20_mad = mad_values.nlargest(20)

# Plotting the top 20 features based on MAD in blue color
plt.figure(figsize=(10, 6))
top_20_mad.plot(kind='barh', color='blue')
plt.title('Top 20 Features by Mean Absolute Difference (MAD)')
plt.xlabel('Mean Absolute Difference')
plt.ylabel('Features')
plt.show()

# Selecting the top 20 features for model training
top_features = top_20_mad.index

X_train, X_test, y_train, y_test = train_test_split(X[top_features], y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_mad = rf.predict(X_test)
accuracy_mad = accuracy_score(y_test, y_pred_mad)
print(f'Accuracy using top 20 MAD features: {accuracy_mad:.4f}')
print(top_20_mad)

Forward Feature Selection

In [ ]:

# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a Forward Feature selection object
# ffs = SequentialFeatureSelector(rf, k_features=20, forward=True, n_jobs=-1)

# # Fit the Sequential Feature Selector to the data
# ffs.fit(X_train, y_train)

# # Get the selected features
# top_20_features = list(ffs.k_feature_names_)

# rf.fit(X_train[top_20_features], y_train)

# # Create a series of features
# feature_importances = pd.Series(rf.feature_importances_, index=top_20_features)

# # Sort features by importance score in descending order
# sorted_feature_importances = feature_importances.sort_values(ascending=False)

# # Plotting the top 20 features based on their importances in blue color
# plt.figure(figsize=(10, 6))
# sorted_feature_importances.plot(kind='barh', color='blue')
# plt.title('Top 20 Features Selected by Sequential Feature Selector')
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.show()

# # Predict on the test data and calculate accuracy using the selected features
# y_pred_ffs = rf.predict(X_test[top_20_features])
# accuracy_ffs = accuracy_score(y_test, y_pred_ffs)

# print("Accuracy using Forward Feature Selection top 20 features:", accuracy_ffs)
# print(sorted_feature_importances)


Backward Feature Selection

In [ ]:
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a Backward Feature Elimination object
# # bfs = SequentialFeatureSelector(rf, k_features='best', forward=False, n_jobs=-1)
# bfs = SequentialFeatureSelector(rf, k_features=20, forward=False, n_jobs=-1)

# # Fit the Sequential Feature Selector to the data
# bfs.fit(X_train, y_train)

# # Get the selected features
# top_20_features_bfs = list(bfs.k_feature_names_)

# rf.fit(X_train[top_20_features_bfs], y_train)

# # Create a series of features importances
# feature_importances_bfs = pd.Series(rf.feature_importances_, index=top_20_features_bfs)

# # Sort features by importance score in descending order
# sorted_feature_importances_bfs = feature_importances_bfs.sort_values(ascending=False)

# # Plotting the top 20 features based on their importances in blue color
# plt.figure(figsize=(10, 6))
# sorted_feature_importances_bfs.plot(kind='barh', color='blue')
# plt.title('Top 20 Features Selected by Backward Feature Elimination')
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.show()

# # Predict on the test data and calculate accuracy using the selected features
# y_pred_bfs = rf.predict(X_test[top_20_features_bfs])
# accuracy_bfs = accuracy_score(y_test, y_pred_bfs)

# print("Accuracy using Backward Feature Elimination top 20 features:", accuracy_bfs)
# print(sorted_feature_importances_bfs)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Create a Backward Feature Elimination object
# bfe = SequentialFeatureSelector(rf, k_features=20, forward=False, n_jobs=-1)

# # Fit the Sequential Feature Selector to the data
# bfe.fit(X_train, y_train)

# # Get the selected features
# selected_features = list(bfe.k_feature_names_)

# # Fit the model using the selected features
# rf.fit(X_train[selected_features], y_train)

# # Get feature importances
# feature_importances = pd.Series(rf.feature_importances_, index=selected_features)

# # Sort features by importance score in descending order
# sorted_feature_importances = feature_importances.sort_values(ascending=False)

# # Plotting the top features based on their importances
# plt.figure(figsize=(10, 6))
# sorted_feature_importances.plot(kind='barh', color='blue')
# plt.title('Feature Importances from Backward Feature Elimination')
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.show()

# # Predict on the test data
# y_pred_bfe = rf.predict(X_test[selected_features])
# accuracy_bfe = accuracy_score(y_test, y_pred_bfe)

# # Calculate the accuracy with selected features
# print("Accuracy using Backward Feature Elimination:", accuracy_bfe)
# print("Selected Features by Backward Feature Elimination:")
# print(sorted_feature_importances)


Recursive Feature Elimination (RFE)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Recursive Feature Elimination object
rfe = RFE(estimator=rf, n_features_to_select=20)

# Fit the RFE object to the data
rfe.fit(X_train, y_train)

# Get the selected features
top_20_features_rfe = X_train.columns[rfe.support_]

rf.fit(X_train[top_20_features_rfe], y_train)

# Create a series of feature importances
feature_importances_rfe = pd.Series(rf.feature_importances_, index=top_20_features_rfe)

# Sort features by importance score in descending order
sorted_feature_importances_rfe = feature_importances_rfe.sort_values(ascending=False)

# Plotting the top 20 features based on their importances in blue color
plt.figure(figsize=(10, 6))
sorted_feature_importances_rfe.plot(kind='barh', color='blue')
plt.title('Top 20 Features Selected by Recursive Feature Elimination (RFE)')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()

# Predict on the test data and calculate accuracy using the selected features
y_pred_rfe = rf.predict(X_test[top_20_features_rfe])
accuracy_rfe = accuracy_score(y_test, y_pred_rfe)

print("Accuracy using Recursive Feature Elimination top 20 features:", accuracy_rfe)
print(sorted_feature_importances_rfe)


Random Forest Importance

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
importances = rf.feature_importances_

# Create a DataFrame for feature importances
feature_importances = pd.Series(importances, index=X.columns)

# Select the top 20 features
top_20_features_rffi = feature_importances.nlargest(20)
top_20_features_names = top_20_features_rffi.index

# Plotting the top 20 feature importances
plt.figure(figsize=(10, 6))
top_20_features_rffi.plot(kind='barh', color='blue')
plt.title('Top 20 Feature Importances from Random Forest')
plt.xlabel('Importance Score')
plt.ylabel('Features')
# plt.axvline(0, color='grey', linestyle='--')  # Optional: add a vertical line at x=0
plt.show()

# Predict on the test data and calculate accuracy using the selected features
# Select the top 20 features (by column names)
top_20_feature_names = top_20_features_rffi.index

rf.fit(X_train[top_20_feature_names], y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_rffi = rf.predict(X_test[top_20_feature_names])
accuracy_rffi = accuracy_score(y_test, y_pred_rffi)

print("Accuracy using Random Forest Importance top 20 features:", accuracy_rffi)
print(top_20_features_rffi)


mRMR

In [ ]:
# Using mRMR to select the top 20 features
top_features = mrmr_classif(X=X, y=y, K=20)

X_train, X_test, y_train, y_test = train_test_split(X[top_features], y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances specifically for the mRMR selected features
importances = rf.feature_importances_

# Create a DataFrame for feature importances
feature_importances = pd.Series(importances, index=top_features)

# Sort by importance score
sorted_importances = feature_importances.sort_values(ascending=False)

# Plotting the feature importances
plt.figure(figsize=(10, 6))
sorted_importances.plot(kind='barh', color='blue')
plt.title('Feature Importances from Random Forest (Selected by mRMR)')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()

# Predict on the test data and calculate accuracy using the selected features
y_pred_mrmr = rf.predict(X_test)
accuracy_mrmr = accuracy_score(y_test, y_pred_mrmr)

print("Accuracy using Random Forest with mRMR selected features:", accuracy_mrmr)
print("Top 20 Features Selected by mRMR with Importances:")
print(sorted_importances)


In [ ]:
# Create a DataFrame for the selected features (with rank)
importance_df = pd.DataFrame({'Feature': top_features, 'Rank': range(1, len(top_features) + 1)})

# Plotting the selected features based on their rank
plt.figure(figsize=(10, 6))
# plt.barh(importance_df['Feature'], importance_df['Rank'], color='steelblue')
importance_df.plot(kind='barh', color='blue')
plt.title('Top 20 Features Selected by mRMR (Rank)')
plt.xlabel('Rank')
plt.ylabel('Features')
plt.show()

# Print sorted selected features
print("Top 20 Features Selected by mRMR (rank):")
print(importance_df)

Lasso Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Lasso model with cross-validation to select features
lasso = LassoCV(alphas=np.logspace(-4, 0, 50), cv=5, random_state=42)
lasso.fit(X_train, y_train)

# Get the coefficients and select features with non-zero coefficients
coef = lasso.coef_
selected_features = np.where(coef != 0)[0]

# Create a DataFrame for feature importances
feature_importances = pd.Series(np.abs(coef[selected_features]), index=X.columns[selected_features])

# Sort by importance score
sorted_importances = feature_importances.sort_values(ascending=False)

# Plotting the feature importances
plt.figure(figsize=(10, 6))
sorted_importances.plot(kind='barh', color='blue')
plt.title('Feature Importances from Lasso Regression')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train.iloc[:, selected_features], y_train)

# Predict on the test data and calculate accuracy using the selected features
y_pred_lasso = rf.predict(X_test.iloc[:, selected_features])
accuracy_lasso = accuracy_score(y_test, y_pred_lasso)

print("Accuracy using Random Forest with Lasso selected features:", accuracy_lasso)
print("Top Features Selected by Lasso with Importances:")
print(sorted_importances)
